**Вклучување на потребните библиотеки**

In [ ]:
import numpy
import sys
import nltk
import string, os 
import numpy as np 
from random import randint

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

**Вчитување на документот со песни**

In [ ]:
file = open("pesni.txt").read()

In [ ]:
file


'Громови на душа\nМагла пак ме покрива\nМириса на невоља\nНоќва сум ти така сам\nУлици накиснати\nПрозорци расплакани\nКаде ли си бар да знам ?\nОд небо нек се спуштаат\nИ громови на душа\nСамо да ти слушнам глас\nПризнавам пред светот\nЌе го минам векот\nДа те најдам ради нас\nНе можам\nОд тебе да оздравам\nСета љубов дур сум жив\nНа тебе ја потрошив\nСветот мој\nМинус тебе во него\nОна што ќе остане\nМалку е\nОд небо нек се спуштаат\nИ громови на душа\nСамо да ти слушнам глас\nПризнавам пред светот\nЌе го минам векот\nДа те најдам ради нас\nНе можам\nОд тебе да оздравам\nСета љубов дур сум жив\nНа тебе ја потрошив\nСветот мој\nМинус тебе во него\nОна што ќе остане\nМалку е\nНе можам\nОд тебе да оздравам\nСета љубов дур сум жив\nНа тебе ја потрошив\nНе можам\nОд тебе да оздравам\nСета љубов дур сум жив\nНа тебе ја потрошив\nСветот мој\nМинус тебе во него\nОна што ќе остане\nМалку е\nМалку е\nПо тебе\nФрли поглед преку рамо ако одиш\nСакам да ти бидам патот кој те води\nКако суво лисје

**Функција за отстранување на интерпункциски знаци и претварање на сите букви во мали**

In [ ]:
def clean_doc(txt):
  txt = "".join(v for v in txt if v not in string.punctuation).lower()
  return txt

**Правење на корпус од стихови за тренирање на моделот**

In [ ]:
clean_file = clean_doc(file)
corpus=clean_file.split('\n')

In [ ]:
print("Дел од корпусот: \n")
corpus[0:5]


Дел од корпусот: 



['громови на душа',
 'магла пак ме покрива',
 'мириса на невоља',
 'ноќва сум ти така сам',
 'улици накиснати']

**Функција за токенизација на корпусот од стихови**

In [ ]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    # токенизирање
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    # претворање на податоците во секвенци од токени
    input_sequences = []

    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]  #momentalniot stih pretvoren vo brojki

        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]                #sekoe negovo podmnozestvo se dodava vo listata input_sequences
            input_sequences.append(n_gram_sequence) 

    return input_sequences, total_words 

In [ ]:
inp_sequences, tot_words = get_sequence_of_tokens(corpus)
print("Број на различни зборови во корпусот: ",tot_words)


Број на различни зборови во корпусот:  1755


**Функција за поставување на иста должина за сите секвенци**

In [ ]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])  # 13 e max dolzina na sekvenca
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=tot_words)
    return predictors, label, max_sequence_len

In [ ]:
predictors, label, max_seq_len = generate_padded_sequences(inp_sequences)

In [ ]:
print("Најдолгиот стих се состои од ",max_seq_len," токени, и воедно таа ќе биде должината на сите секвенци од токени")
print("Првата секвенца поделена на: \npredictors: ",predictors[0],"\nlabel: ",label[0])

Најдолгиот стих се состои од  13  токени, и воедно таа ќе биде должината на сите секвенци од токени
Првата секвенца поделена на: 
predictors:  [  0   0   0   0   0   0   0   0   0   0   0 460] 
label:  [0. 0. 0. ... 0. 0. 0.]


**Креирање на моделот**

In [ ]:
input_len = max_seq_len - 1
model = Sequential()
   
# Add Input Embedding Layer
model.add(Embedding(tot_words, 5, input_length=input_len))
  
# Add Hidden Layers
model.add(GRU(80, return_sequences = True))
model.add(GRU(120))
model.add(Dropout(0.2))
    
# Add Output Layer
model.add(Dense(tot_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
earlystop = EarlyStopping(monitor='loss', patience=5, verbose=0, mode='auto')

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 12, 5)             8775      
_________________________________________________________________
gru (GRU)                    (None, 12, 80)            20880     
_________________________________________________________________
gru_1 (GRU)                  (None, 120)               72720     
_________________________________________________________________
dropout (Dropout)            (None, 120)               0         
_________________________________________________________________
dense (Dense)                (None, 1755)              212355    
Total params: 314,730
Trainable params: 314,730
Non-trainable params: 0
_________________________________________________________________


**Тренирање на моделот**

In [ ]:
model.fit(predictors, label, epochs=400, verbose=2, callbacks=[earlystop])

Epoch 1/400
238/238 - 10s - loss: 6.6092 - accuracy: 0.0427
Epoch 2/400
238/238 - 5s - loss: 6.1664 - accuracy: 0.0441
Epoch 3/400
238/238 - 5s - loss: 6.0174 - accuracy: 0.0438
Epoch 4/400
238/238 - 6s - loss: 5.8406 - accuracy: 0.0482
Epoch 5/400
238/238 - 5s - loss: 5.6122 - accuracy: 0.0540
Epoch 6/400
238/238 - 5s - loss: 5.3779 - accuracy: 0.0637
Epoch 7/400
238/238 - 5s - loss: 5.1527 - accuracy: 0.0715
Epoch 8/400
238/238 - 5s - loss: 4.9344 - accuracy: 0.0895
Epoch 9/400
238/238 - 5s - loss: 4.7167 - accuracy: 0.1109
Epoch 10/400
238/238 - 5s - loss: 4.5230 - accuracy: 0.1310
Epoch 11/400
238/238 - 5s - loss: 4.3477 - accuracy: 0.1487
Epoch 12/400
238/238 - 5s - loss: 4.1786 - accuracy: 0.1738
Epoch 13/400
238/238 - 5s - loss: 4.0384 - accuracy: 0.1941
Epoch 14/400
238/238 - 5s - loss: 3.9033 - accuracy: 0.2102
Epoch 15/400
238/238 - 5s - loss: 3.7827 - accuracy: 0.2252
Epoch 16/400
238/238 - 5s - loss: 3.6666 - accuracy: 0.2465
Epoch 17/400
238/238 - 5s - loss: 3.5660 - accur

**Правење на корпус од зборови за избирање рандом почетен збор**

In [ ]:
seed_word = clean_file
seed_word = seed_word.replace("\n", " ")
seed_word = seed_word.replace("   ", " ")
seed_word = seed_word.replace("  ", " ")
seed_word = seed_word.replace("    ", " ")
seed_word= seed_word.split(' ')

**Функција за генерирање на текст**

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for i in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0] #random zborot go pretvara vo brojka(token)
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre') #se doveduva do dolzina 12
        predicted = model.predict_classes(token_list, verbose=0)  # se predviduva eden zbor (vo predicted e negovata brojka)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word  # (vo output_word e samiot zbor)
                break

        seed_text += " "+output_word  #toj se dodava na seed-ot
    return seed_text

**Генерирање на песната**

In [ ]:
for j in range(0, 4): # 4 strofi
  seed_text = seed_word[randint(0, len(seed_word))]

  text = generate_text(seed_text, 6, model, max_seq_len)
  print(text.rsplit(' ', 1)[0])  

  for i in range(0, 3): # 4 stiha vo strofa
    words = text.split()
    lastWord = words[len(words)-1]
    text = generate_text(lastWord, 6, model, max_seq_len)
    print(text.rsplit(' ', 1)[0])

  print('\n')

рефрен отварам да ме проголта сонот
за овој свет за овој свет
до мене да си и мирно
спиеш во гитарите ќе горат блиску


стих моја дива јагода ли ме
продаде во образ солза сува да
в постела гушнати излишни се зборови
скриена си слатка но и студена


знам дојде крај крај се спуштаат
пепел да ми признаеш судбина знам
и со детска насмевка и со
детска овој свет за овој свет


каде ли беше толку години кога
солзи прават златен прстен сонце бури
смислуваш ме гризе малечка но ден
губам ако смееш е разбудам секој


